<a href="https://colab.research.google.com/github/contactkit/ANN1-ai1-fall2019/blob/master/Homework4_ipnyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils

In [0]:
!pip install keras

In [0]:
!pip install wandb

     |████████████████████████████████| 1.4MB 3.3MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 460kB 19.2MB/s 
     |████████████████████████████████| 102kB 12.2MB/s 
     |████████████████████████████████| 71kB 9.1MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.1-cp36-none-any.whl size=73503 sha256=71036edab5a47adebe6642ece0335c2e691c106aefc6c689919a24b2cbdcdce6
  Stored in directory: /root/.cache/pip/wheels/d3/07/ac/f8e4e29b1f6df6ed9569891d26a326f2af9ff8b77448cc46b8
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=371b930d82a3f9cc1c260b226c4306cc67f2b23ebd623e27ee6bc0833d94df8e
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=5e

In [0]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 07ec05e4ac815f80c08a6ba0d5451a511a9084d9
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
from wandb.keras import WandbCallback
import wandb
import os

In [0]:
run = wandb.init()
config = run.config
config.first_layer_convs = 64
config.first_layer_conv_width = 5
config.first_layer_conv_height = 5
config.img_width = 32
config.img_height = 32
config.epochs = 20

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 3s 0us/step


In [0]:
X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

In [0]:
X_train.shape

(50000, 32, 32, 3)

In [0]:
X_train = X_train.reshape(X_train.shape[0], config.img_width, config.img_height, 3)
X_test = X_test.reshape(X_test.shape[0], config.img_width, config.img_height, 3)

In [0]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
labels = range(10)

In [0]:
model = Sequential()
model.add(Conv2D(64,(config.first_layer_conv_width, config.first_layer_conv_height),input_shape=(32,32,3),padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model.add(Conv2D(192,(5,5),padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model.add(Conv2D(384,(3,3),padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model.add(Conv2D(256,(5,5),padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model.add(Conv2D(256,(5,5),padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(256, activation='relu'))

In [0]:
model.add(Dense(4096, activation='relu'))

In [0]:
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 64)        4864      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 192)       307392    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 192)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 384)         663936    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 4, 4, 384)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 256)        

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [0]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=config.epochs,
          callbacks=[WandbCallback(data_type="image", save_model=False)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 50000 samples, validate on 10000 samples
Epoch 1/20





50000/50000 [==============================] - 45s 899us/step - loss: 1.7382 - acc: 0.3390 - val_loss: 1.4149 - val_acc: 0.4844
Epoch 2/20
50000/50000 [==============================] - 29s 590us/step - loss: 1.2481 - acc: 0.5477 - val_loss: 1.1307 - val_acc: 0.5929
Epoch 3/20
50000/50000 [==============================] - 30s 593us/step - loss: 1.0437 - acc: 0.6298 - val_loss: 1.0070 - val_acc: 0.6495
Epoch 4/20
50000/50000 [==============================] - 30s 594us/step - loss: 0.9048 - acc: 0.6839 - val_loss: 0.9665 - val_acc: 0.6717
Epoch 5/20
50000/50000 [==============================] - 30s 595us/step - loss: 0.7970 - acc: 0.7224 - val_loss: 0.9469 - val_acc: 0.6864
Epoch 6/20
50000/50000 [==============================] - 30s 595us/step - loss: 0.7019 - acc: 0.7588 - val_loss: 0.9438 - val_acc: 0.6860
Epoch 7/20
500

In [0]:
model1 = Sequential()
model1.add(Conv2D(64,(config.first_layer_conv_width, config.first_layer_conv_height),input_shape=(32,32,3),padding="same",activation='relu'))
model1.add(Dropout(0.2))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model1.add(Conv2D(192,(5,5),padding="same",activation='relu'))
model1.add(Dropout(0.2))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model1.add(Conv2D(384,(3,3),padding="same",activation='relu'))
model1.add(Dropout(0.2))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model1.add(Conv2D(256,(5,5),padding="same",activation='relu'))
model1.add(Dropout(0.2))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model1.add(Conv2D(256,(5,5),padding="same",activation='relu'))
model1.add(Dropout(0.2))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model1.add(Flatten())

In [0]:
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.5))

In [0]:
model1.add(Dense(4096, activation='relu'))
model1.add(Dropout(0.5))

In [0]:
model1.add(Dense(num_classes, activation='softmax'))

In [0]:
model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 32, 32, 64)        4864      
_________________________________________________________________
dropout_18 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 16, 16, 192)       307392    
_________________________________________________________________
dropout_19 (Dropout)         (None, 16, 16, 192)       0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 8, 8, 192)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 8, 8, 384)        

In [0]:
model1.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [0]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=config.epochs,
          callbacks=[WandbCallback(data_type="image", save_model=False)])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
  256/50000 [..............................] - ETA: 4:39 - loss: 2.3189 - acc: 0.0898

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.102173). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


50000/50000 [==============================] - 35s 697us/step - loss: 1.9207 - acc: 0.2620 - val_loss: 1.8704 - val_acc: 0.3315
Epoch 2/20
50000/50000 [==============================] - 34s 670us/step - loss: 1.5303 - acc: 0.4393 - val_loss: 1.4981 - val_acc: 0.4777
Epoch 3/20
50000/50000 [==============================] - 33s 666us/step - loss: 1.3999 - acc: 0.4975 - val_loss: 1.3913 - val_acc: 0.5170
Epoch 4/20
50000/50000 [==============================] - 33s 668us/step - loss: 1.3187 - acc: 0.5329 - val_loss: 1.3216 - val_acc: 0.5513
Epoch 5/20
50000/50000 [==============================] - 33s 667us/step - loss: 1.2602 - acc: 0.5575 - val_loss: 1.3529 - val_acc: 0.5464
Epoch 6/20
50000/50000 [==============================] - 33s 667us/step - loss: 1.2033 - acc: 0.5812 - val_loss: 1.2204 - val_acc: 0.5730
Epoch 7/20
50000/50000 [==============================] - 33s 667us/step - loss: 1.1724 - acc: 0.5940 - val_loss: 1.2442 - val_acc: 0.5975
Epoch 8/20
50000/50000 [==============

In [0]:
from keras.layers import BatchNormalization
model2 = Sequential()
model2.add(Conv2D(64,(config.first_layer_conv_width, config.first_layer_conv_height),input_shape=(32,32,3),padding="same",activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model2.add(Conv2D(192,(5,5),padding="same",activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model2.add(Conv2D(384,(3,3),padding="same",activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model2.add(Conv2D(256,(5,5),padding="same",activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model2.add(Conv2D(256,(5,5),padding="same",activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.2))
model2.add(MaxPooling2D(pool_size=(2, 2),strides=2))

In [0]:
model2.add(Flatten())

In [0]:
model2.add(Dense(256, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))

In [0]:
model2.add(Dense(4096, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))

In [0]:
model2.add(Dense(num_classes, activation='softmax'))

In [0]:
model2.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 32, 32, 64)        4864      
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_33 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 16, 16, 192)       307392    
_________________________________________________________________
batch_normalization_10 (Batc (None, 16, 16, 192)       768       
_________________________________________________________________
dropout_34 (Dropout)         (None, 16, 16, 192)      

In [0]:
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [0]:
model2.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=config.epochs,
          callbacks=[WandbCallback(data_type="image", save_model=False)])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
  256/50000 [..............................] - ETA: 9:28 - loss: 3.8560 - acc: 0.1289 

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.107111). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


50000/50000 [==============================] - 44s 871us/step - loss: 2.7341 - acc: 0.2843 - val_loss: 2.6333 - val_acc: 0.2082
Epoch 2/20
50000/50000 [==============================] - 41s 811us/step - loss: 2.2585 - acc: 0.3398 - val_loss: 2.5416 - val_acc: 0.2800
Epoch 3/20
50000/50000 [==============================] - 40s 802us/step - loss: 2.2667 - acc: 0.3514 - val_loss: 2.5256 - val_acc: 0.3130
Epoch 4/20
50000/50000 [==============================] - 40s 805us/step - loss: 2.3728 - acc: 0.3195 - val_loss: 4.2827 - val_acc: 0.2871
Epoch 5/20
50000/50000 [==============================] - 40s 805us/step - loss: 1.9632 - acc: 0.3379 - val_loss: 2.2403 - val_acc: 0.3579
Epoch 6/20
50000/50000 [==============================] - 40s 805us/step - loss: 1.5926 - acc: 0.4273 - val_loss: 2.4279 - val_acc: 0.3750
Epoch 7/20
50000/50000 [==============================] - 40s 803us/step - loss: 1.3919 - acc: 0.5036 - val_loss: 2.1306 - val_acc: 0.4555
Epoch 8/20
50000/50000 [==============

In [0]:
from keras.layers import BatchNormalization
model3 = Sequential()
model3.add(Conv2D(64,(config.first_layer_conv_width, config.first_layer_conv_height),input_shape=(32,32,3),padding="same",activation='relu',strides=2))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

In [0]:
model3.add(Conv2D(192,(5,5),padding="same",activation='relu',strides=2))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

In [0]:
model3.add(Conv2D(384,(3,3),padding="same",activation='relu',strides=2))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

In [0]:
model3.add(Conv2D(256,(5,5),padding="same",activation='relu',strides=2))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

In [0]:
model3.add(Conv2D(256,(5,5),padding="same",activation='relu',strides=2))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

In [0]:
model3.add(Conv2D(4096,(1,1),activation='relu',strides=1))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))

In [0]:
model3.add(Conv2D(4096,(1,1), activation='relu',strides=1))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))

In [0]:
model3.add(Conv2D(num_classes,(1,1), activation='softmax',strides=1))

In [0]:
model3.add(Flatten())

In [0]:
model3.add(Dense(num_classes, activation='softmax'))

In [0]:
model3.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 16, 16, 64)        4864      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 192)         307392    
_________________________________________________________________
batch_normalization_3 (Batch (None, 8, 8, 192)         768       
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 8, 192)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 384)        

In [0]:
model3.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [0]:
model3.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=config.epochs,
          callbacks=[WandbCallback(data_type="image", save_model=False)])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 72s 1ms/step - loss: 2.2075 - acc: 0.1649 - val_loss: 2.2045 - val_acc: 0.1727
Epoch 2/20
50000/50000 [==============================] - 64s 1ms/step - loss: 2.1532 - acc: 0.1772 - val_loss: 2.0985 - val_acc: 0.1931
Epoch 3/20
50000/50000 [==============================] - 64s 1ms/step - loss: 2.0964 - acc: 0.1833 - val_loss: 2.0833 - val_acc: 0.1970
Epoch 4/20
50000/50000 [==============================] - 64s 1ms/step - loss: 2.0794 - acc: 0.1886 - val_loss: 2.1704 - val_acc: 0.1826
Epoch 5/20
50000/50000 [==============================] - 64s 1ms/step - loss: 2.0726 - acc: 0.1902 - val_loss: 2.0516 - val_acc: 0.2153
Epoch 6/20
50000/50000 [==============================] - 64s 1ms/step - loss: 2.0520 - acc: 0.2030 - val_loss: 2.0571 - val_acc: 0.1996
Epoch 7/20
50000/50000 [========

In [0]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
datagen.fit(x_train)

In [0]:
model3.fit_generator(datagen.flow(x_train, y_train, batch_size=256),validation_data=(x_test,y_test),
                    steps_per_epoch=len(x_train) /256, epochs=20)

Epoch 1/20
196/195 [==============================] - 31s 160ms/step - loss: 2.1542 - acc: 0.1898 - val_loss: 2.7415 - val_acc: 0.1001
Epoch 2/20
196/195 [==============================] - 27s 140ms/step - loss: 2.0758 - acc: 0.2041 - val_loss: 2.6531 - val_acc: 0.1040
Epoch 3/20
196/195 [==============================] - 27s 139ms/step - loss: 2.0468 - acc: 0.2083 - val_loss: 2.7150 - val_acc: 0.1003
Epoch 4/20
196/195 [==============================] - 27s 138ms/step - loss: 2.0394 - acc: 0.2104 - val_loss: 2.7231 - val_acc: 0.1000
Epoch 5/20
196/195 [==============================] - 27s 140ms/step - loss: 2.0315 - acc: 0.2115 - val_loss: 2.7403 - val_acc: 0.1000
Epoch 6/20
196/195 [==============================] - 27s 138ms/step - loss: 2.0248 - acc: 0.2117 - val_loss: 2.7512 - val_acc: 0.1009
Epoch 7/20
196/195 [==============================] - 27s 139ms/step - loss: 2.0258 - acc: 0.2165 - val_loss: 2.7609 - val_acc: 0.1016
Epoch 8/20
196/195 [==============================] - 2

In [0]:
model3.fit_generator(datagen.flow(x_train, y_train, batch_size=256),validation_data=(x_test,y_test),
                    steps_per_epoch=len(x_train) /256, epochs=200)

Epoch 1/200
196/195 [==============================] - 28s 145ms/step - loss: 2.0043 - acc: 0.2192 - val_loss: 2.2994 - val_acc: 0.1586
Epoch 2/200
196/195 [==============================] - 28s 140ms/step - loss: 2.0060 - acc: 0.2207 - val_loss: 2.8085 - val_acc: 0.1000
Epoch 3/200
196/195 [==============================] - 28s 141ms/step - loss: 2.0044 - acc: 0.2245 - val_loss: 2.8497 - val_acc: 0.1000
Epoch 4/200
196/195 [==============================] - 28s 140ms/step - loss: 1.9910 - acc: 0.2258 - val_loss: 2.8951 - val_acc: 0.1001
Epoch 5/200
196/195 [==============================] - 27s 140ms/step - loss: 1.9871 - acc: 0.2253 - val_loss: 2.2932 - val_acc: 0.1623
Epoch 6/200
196/195 [==============================] - 28s 141ms/step - loss: 1.9811 - acc: 0.2256 - val_loss: 2.8511 - val_acc: 0.1014
Epoch 7/200
196/195 [==============================] - 28s 141ms/step - loss: 1.9835 - acc: 0.2227 - val_loss: 2.8357 - val_acc: 0.1043
Epoch 8/200
196/195 [===========================